# Chapter 2: Introduction to Data Analysis with Scala Spark

In this notebook, we include the exploratory analysis step corresponding to the Chapter 2 of "Advanced Anallytics with Spark".

In [41]:
import org.apache.spark.sql.functions.{col, first}
val sparkBis = spark
import sparkBis.implicits._
import org.apache.spark.sql.DataFrame

sparkBis = org.apache.spark.sql.SparkSession@3a4b2d7c


Spark local-1536004739511: Some(http://01f0163a8d01:4040)

## Data Loading

The first step is to load the data. We will do it in two steps, as we have to configure some options like the header, possible null values, etc. In addition, we will only load a sample of the data, leaving the processing of the whole data for other steps of the data pipeline process (deployment in production).

In [2]:
val data_path = "../data/linkage/"

data_path = ../data/linkage/


../data/linkage/

In [3]:
val preview = spark.read.csv(data_path).sample(fraction = 0.001, seed = 42)

preview = [_c0: string, _c1: string ... 10 more fields]


[_c0: string, _c1: string ... 10 more fields]

In [4]:
preview.show()

+-----+-----+-----------------+-----+-----------------+---+---+---+---+---+----+-----+
|  _c0|  _c1|              _c2|  _c3|              _c4|_c5|_c6|_c7|_c8|_c9|_c10| _c11|
+-----+-----+-----------------+-----+-----------------+---+---+---+---+---+----+-----+
|26438|66389|                1|0.125|                1|  ?|  1|  1|  1|  1|   1| TRUE|
|39365|45304|                1|    ?|            0.125|  ?|  1|  1|  0|  0|   0|FALSE|
|32904|98672|                1|    ?|              0.6|  ?|  1|  0|  0|  1|   0|FALSE|
|18891|28561|                1|    ?|0.428571428571429|  ?|  1|  0|  1|  0|   0|FALSE|
|20958|68948|             0.25|    ?|             0.25|  ?|  1|  0|  0|  0|   0|FALSE|
| 1474|86724|                1|    ?|              0.1|  ?|  1|  1|  0|  0|   0|FALSE|
|27349|38989|                0|    ?|                1|  ?|  1|  0|  0|  0|   0|FALSE|
|17375|49955|                1|    ?|0.142857142857143|  ?|  1|  1|  0|  0|   0|FALSE|
|23859|95295|                1|    ?|      

Options:
* option: header = true
* option: nullValue = "?"
* option: inferSchema = "true"

In [5]:
val data = spark.read.option("header", "true")
    .option("nullValue", "?")
    .option("inferSchema", "true").csv(data_path).sample(fraction = 0.001, seed = 42)

data = [id_1: int, id_2: int ... 10 more fields]


[id_1: int, id_2: int ... 10 more fields]

In [6]:
data.cache()
data.count()

5819

In [7]:
data.show()

+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|     cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
|35218|38094|              1.0|        null|              1.0|        null|      1|     1|     1|     1|      1|    true|
|38563|88351|              1.0|        null|              0.0|        null|      1|     1|     0|     0|      0|   false|
|56678|66201|              1.0|        null|              0.0|        null|      1|     0|     0|     1|      0|   false|
|64906|93890|              1.0|        null|             0.25|        null|      1|     1|     0|     0|      0|   false|
| 4870|34116|              1.0|        null|              0.0|        null|      1|     0|     1|     0|      0|   false|
|45483|69150|0.166666666

In [8]:
data.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



## Exploratory Analysis

Let's now perform some preliminary analysis. We first check the distribution of our target variable, `is_match`.

In [9]:
data.groupBy("is_match").count().orderBy(col("count").desc).show()

+--------+-----+
|is_match|count|
+--------+-----+
|   false| 5796|
|    true|   23|
+--------+-----+



We can also now check the summary of the data, grouping the results as a function of the possible values of the `is_match` column.

In [10]:
val summary = data.describe()
summary.show()

+-------+------------------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+--------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|     cmp_fname_c2|      cmp_lname_c1|       cmp_lname_c2|           cmp_sex|            cmp_bd|             cmp_bm|             cmp_by|             cmp_plz|
+-------+------------------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+--------------------+
|  count|              5819|              5819|              5816|               95|              5819|                  2|              5819|              5817|               5817|               5817|                5804|
|   mean| 33414.59133871799|  66340.5959786905|0.7131791857462408|0.924093567251462|0.3197607897534861|  0.1

summary = [summary: string, id_1: string ... 10 more fields]


[summary: string, id_1: string ... 10 more fields]

In [11]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+------------------+-----------------+
|summary|      cmp_fname_c1|     cmp_fname_c2|
+-------+------------------+-----------------+
|  count|              5816|               95|
|   mean|0.7131791857462408|0.924093567251462|
| stddev|0.3890878881040213|0.238267300087884|
|    min|               0.0|              0.0|
|    max|               1.0|              1.0|
+-------+------------------+-----------------+



## Data Preparation

In this section, we are going to calculate, per each group of `is_match`, the main statistics (count, mean, stdev, etc...) of all the variables, and we will display it in a wide format, where the rows correspond to each variable and the columns to each statistics.

We use first the `describe()` functionality of Spark.

In [12]:
val matches = data.filter(col("is_match") === true)
val misses = data.filter(col("is_match") === false)
val matchSummary = matches.describe()
val missSummary = misses.describe()

matches = [id_1: int, id_2: int ... 10 more fields]
misses = [id_1: int, id_2: int ... 10 more fields]
matchSummary = [summary: string, id_1: string ... 10 more fields]
missSummary = [summary: string, id_1: string ... 10 more fields]


[summary: string, id_1: string ... 10 more fields]

In [13]:
matchSummary.printSchema()

root
 |-- summary: string (nullable = true)
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: string (nullable = true)
 |-- cmp_plz: string (nullable = true)



Now we transform the previous match data into long format.

In [23]:
val matchSchema = matchSummary.schema
val longDFMatch = matchSummary.rdd.flatMap(row => {
    val metricName = row.getString(0)
    (1 until row.size).map(i => (metricName, matchSchema(i).name, {row.getString(i) match {
        case value: String => value
        case null => null
    }}))
}).toDF("metric", "field", "value")

matchSchema = StructType(StructField(summary,StringType,true), StructField(id_1,StringType,true), StructField(id_2,StringType,true), StructField(cmp_fname_c1,StringType,true), StructField(cmp_fname_c2,StringType,true), StructField(cmp_lname_c1,StringType,true), StructField(cmp_lname_c2,StringType,true), StructField(cmp_sex,StringType,true), StructField(cmp_bd,StringType,true), StructField(cmp_bm,StringType,true), StructField(cmp_by,StringType,true), StructField(cmp_plz,StringType,true))
longDFMatch = [metric: string, field: string ... 1 more field]


[metric: string, field: string ... 1 more field]

In [24]:
longDFMatch.show()

+------+------------+------------------+
|metric|       field|             value|
+------+------------+------------------+
| count|        id_1|                23|
| count|        id_2|                23|
| count|cmp_fname_c1|                23|
| count|cmp_fname_c2|                 2|
| count|cmp_lname_c1|                23|
| count|cmp_lname_c2|                 0|
| count|     cmp_sex|                23|
| count|      cmp_bd|                23|
| count|      cmp_bm|                23|
| count|      cmp_by|                23|
| count|     cmp_plz|                23|
|  mean|        id_1|           37772.0|
|  mean|        id_2|50867.434782608696|
|  mean|cmp_fname_c1|               1.0|
|  mean|cmp_fname_c2|               1.0|
|  mean|cmp_lname_c1|               1.0|
|  mean|cmp_lname_c2|              null|
|  mean|     cmp_sex|0.9565217391304348|
|  mean|      cmp_bd|0.9565217391304348|
|  mean|      cmp_bm|               1.0|
+------+------------+------------------+
only showing top

And now we pivot it so the data is arranged in the desired format.

In [25]:
val wideDFMatch = longDFMatch.groupBy("field")
    .pivot("metric", Seq("count", "mean", "stddev", "max", "min"))
    .agg(first("value"))

wideDFMatch = [field: string, count: string ... 4 more fields]


[field: string, count: string ... 4 more fields]

In [26]:
wideDFMatch.show()

+------------+-----+------------------+-------------------+-----+----+
|       field|count|              mean|             stddev|  max| min|
+------------+-----+------------------+-------------------+-----+----+
|        id_2|   23|50867.434782608696| 21539.826208098773|88420|3662|
|     cmp_plz|   23|               1.0|                0.0|    1|   1|
|cmp_lname_c1|   23|               1.0|                0.0|  1.0| 1.0|
|cmp_lname_c2|    0|              null|               null| null|null|
|     cmp_sex|   23|0.9565217391304348|0.20851441405707477|    1|   0|
|      cmp_bm|   23|               1.0|                0.0|    1|   1|
|cmp_fname_c2|    2|               1.0|                0.0|  1.0| 1.0|
|cmp_fname_c1|   23|               1.0|                0.0|  1.0| 1.0|
|        id_1|   23|           37772.0|  18529.77306194743|71241|3029|
|      cmp_bd|   23|0.9565217391304348|0.20851441405707477|    1|   0|
|      cmp_by|   23|0.9565217391304348|0.20851441405707477|    1|   0|
+-----

Now, we have to do the same for the `miss` data. We are going to build a function that performs the previous transformations so we can reuse it in the future.

In [27]:
def getLongSummary(summary: DataFrame): DataFrame = {
    
    val schema = summary.schema
    val longDF = summary.rdd.flatMap(row => {
        val metricName = row.getString(0)
        (1 until row.size).map(i => (metricName, schema(i).name, {row.getString(i) match {
            case value: String => value
            case null => null
        }}))
    }).toDF("metric", "field", "value")
    
    longDF
    
}


def getPivotSummary(summary: DataFrame): DataFrame = {
    
    val longDF = getLongSummary(summary)
    
    val wideDF = longDF.groupBy("field")
    .pivot("metric", Seq("count", "mean", "stddev", "max", "min"))
    .agg(first("value"))
    
    wideDF
    
}

getLongSummary: (summary: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
getPivotSummary: (summary: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [28]:
val wideDFMatchBis = getPivotSummary(matchSummary)

wideDFMatchBis = [field: string, count: string ... 4 more fields]


[field: string, count: string ... 4 more fields]

In [29]:
wideDFMatchBis.show()

+------------+-----+------------------+-------------------+-----+----+
|       field|count|              mean|             stddev|  max| min|
+------------+-----+------------------+-------------------+-----+----+
|        id_2|   23|50867.434782608696| 21539.826208098773|88420|3662|
|     cmp_plz|   23|               1.0|                0.0|    1|   1|
|cmp_lname_c1|   23|               1.0|                0.0|  1.0| 1.0|
|cmp_lname_c2|    0|              null|               null| null|null|
|     cmp_sex|   23|0.9565217391304348|0.20851441405707477|    1|   0|
|      cmp_bm|   23|               1.0|                0.0|    1|   1|
|cmp_fname_c2|    2|               1.0|                0.0|  1.0| 1.0|
|cmp_fname_c1|   23|               1.0|                0.0|  1.0| 1.0|
|        id_1|   23|           37772.0|  18529.77306194743|71241|3029|
|      cmp_bd|   23|0.9565217391304348|0.20851441405707477|    1|   0|
|      cmp_by|   23|0.9565217391304348|0.20851441405707477|    1|   0|
+-----

In [30]:
val wideDFMiss = getPivotSummary(missSummary)

wideDFMiss = [field: string, count: string ... 4 more fields]


[field: string, count: string ... 4 more fields]

In [31]:
wideDFMiss.show()

+------------+-----+--------------------+-------------------+-----------------+-----+
|       field|count|                mean|             stddev|              max|  min|
+------------+-----+--------------------+-------------------+-----------------+-----+
|        id_2| 5796|   66401.99741200828| 23899.696454240267|            99997|  783|
|     cmp_plz| 5781|0.001729804532087...|0.04155852577779171|                1|    0|
|cmp_lname_c1| 5796| 0.31706142780806346| 0.3349985064290111|              1.0|  0.0|
|cmp_lname_c2|    2|   0.173611111111111|0.06874649261535862|0.222222222222222|0.125|
|     cmp_sex| 5796|  0.9556590752242926| 0.2058691804780139|                1|    0|
|      cmp_bm| 5794|  0.4925785295132896|0.49998806765825293|                1|    0|
|cmp_fname_c2|   93|  0.9224611708482676| 0.2405774619790335|              1.0|  0.0|
|cmp_fname_c1| 5793|  0.7120404184878538| 0.3894387956025427|              1.0|  0.0|
|        id_1| 5796|   33397.30003450656| 23926.886833

Now, we calculate the difference between the mean values of the variables of the `match` and `miss` case.

In [35]:
wideDFMiss.createOrReplaceTempView("miss_desc")
wideDFMatch.createOrReplaceTempView("match_desc")

lastException: Throwable = null


In [36]:
spark.sql("""
    SELECT a.field, a.count + b.count total, a.mean - b.mean delta
    FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
    ORDER BY delta DESC, total DESC
""").show()

+------------+------+--------------------+
|       field| total|               delta|
+------------+------+--------------------+
|        id_1|5819.0|   4374.699965493441|
|     cmp_plz|5804.0|  0.9982701954679122|
|      cmp_by|5817.0|  0.7438879800693371|
|      cmp_bd|5817.0|  0.7335324398553227|
|cmp_lname_c1|5819.0|  0.6829385721919365|
|      cmp_bm|5817.0|  0.5074214704867104|
|cmp_fname_c1|5816.0|  0.2879595815121462|
|cmp_fname_c2|  95.0| 0.07753882915173238|
|     cmp_sex|5819.0|8.626639061422026E-4|
|        id_2|5819.0| -15534.562629399581|
|cmp_lname_c2|   2.0|                null|
+------------+------+--------------------+



Based on the previous data, we select the most important variables (those with a high `total` count and a large `delta`) for our simple classification algorithm (`id_*` should not be included as they are not actually features):

    * cmp_plz
    * cmp_by
    * cmp_bd
    * cmp_lname_c1
    * cmp_bm
    * cmp_fname_c1


In [37]:
data.show()

+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|     cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
|35218|38094|              1.0|        null|              1.0|        null|      1|     1|     1|     1|      1|    true|
|38563|88351|              1.0|        null|              0.0|        null|      1|     1|     0|     0|      0|   false|
|56678|66201|              1.0|        null|              0.0|        null|      1|     0|     0|     1|      0|   false|
|64906|93890|              1.0|        null|             0.25|        null|      1|     1|     0|     0|      0|   false|
| 4870|34116|              1.0|        null|              0.0|        null|      1|     0|     1|     0|      0|   false|
|45483|69150|0.166666666

In [39]:
val data_imputed = data.na.fill(0)

data_imputed = [id_1: int, id_2: int ... 10 more fields]


[id_1: int, id_2: int ... 10 more fields]

In [40]:
data_imputed.show()

+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|     cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
|35218|38094|              1.0|         0.0|              1.0|         0.0|      1|     1|     1|     1|      1|    true|
|38563|88351|              1.0|         0.0|              0.0|         0.0|      1|     1|     0|     0|      0|   false|
|56678|66201|              1.0|         0.0|              0.0|         0.0|      1|     0|     0|     1|      0|   false|
|64906|93890|              1.0|         0.0|             0.25|         0.0|      1|     1|     0|     0|      0|   false|
| 4870|34116|              1.0|         0.0|              0.0|         0.0|      1|     0|     1|     0|      0|   false|
|45483|69150|0.166666666

We will calculate the sum of the previous selected features, and then we will propose a simple classification algorithm based on this value

In [55]:
val data_score = data_imputed.withColumn("score", (col("cmp_plz") + col("cmp_by") + col("cmp_bd") 
                                                   + col("cmp_lname_c1") + col("cmp_bm") 
                                                   + col("cmp_fname_c1"))).select("is_match", "score")

data_score = [is_match: boolean, score: double]


[is_match: boolean, score: double]

In [56]:
data_score.show()

+--------+------------------+
|is_match|             score|
+--------+------------------+
|    true|               6.0|
|   false|               2.0|
|   false|               2.0|
|   false|              2.25|
|   false|               2.0|
|   false| 0.666666666666667|
|   false| 1.166666666666667|
|   false|               1.0|
|   false| 0.982142857142857|
|   false|1.7222222222222219|
|   false|             2.125|
|   false|2.1111111111111107|
|   false| 1.111111111111111|
|   false| 0.857142857142857|
|   false|               1.5|
|   false|               2.0|
|   false|             2.375|
|   false|2.1111111111111107|
|   false|               0.6|
|   false| 1.285714285714286|
+--------+------------------+
only showing top 20 rows



We will check now the performance of our model by imposing a threshold value of 4.0.

In [59]:
val threshold = 4.0
data_score.withColumn("above", col("score") >= threshold).groupBy("above").pivot("is_match", Seq(true, false)).count().show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| true|  23|    5|
|false|null| 5791|
+-----+----+-----+



threshold = 4.0


4.0

As we would like to check different threshold values, we will build a function.

In [62]:
def getScore(data_score: DataFrame, threshold: Double): DataFrame = {
    data_score.withColumn("above", col("score") >= threshold).groupBy("above").pivot("is_match", Seq(true, false)).count()
}

getScore: (data_score: org.apache.spark.sql.DataFrame, threshold: Double)org.apache.spark.sql.DataFrame


In [65]:
val score_5p0 = getScore(data_score, 5.0)
val score_4p5 = getScore(data_score, 4.5)
val score_4p0 = getScore(data_score, 4.0)
val score_3p5 = getScore(data_score, 3.5)
val score_3p0 = getScore(data_score, 3.0)
val score_2p5 = getScore(data_score, 2.5)

score_5p0 = [above: boolean, true: bigint ... 1 more field]
score_4p5 = [above: boolean, true: bigint ... 1 more field]
score_4p0 = [above: boolean, true: bigint ... 1 more field]
score_3p5 = [above: boolean, true: bigint ... 1 more field]
score_3p0 = [above: boolean, true: bigint ... 1 more field]
score_2p5 = [above: boolean, true: bigint ... 1 more field]


[above: boolean, true: bigint ... 1 more field]

In [64]:
score_5p0.show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| true|  22| null|
|false|   1| 5796|
+-----+----+-----+



In [66]:
score_4p5.show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| true|  22| null|
|false|   1| 5796|
+-----+----+-----+



In [67]:
score_4p0.show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| true|  23|    5|
|false|null| 5791|
+-----+----+-----+



In [68]:
score_3p5.show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| true|  23|   57|
|false|null| 5739|
+-----+----+-----+



In [69]:
score_3p0.show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| true|  23|  478|
|false|null| 5318|
+-----+----+-----+



In [70]:
score_2p5.show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| true|  23|  573|
|false|null| 5223|
+-----+----+-----+

